<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Talk_to_datasets_worldbank_project_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required dependencies

In [1]:
!pip install -qU langchain huggingface_hub Chromadb pyPDF sentence-transformers accelerate ctransformers datasets openai tiktoken lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.

## Import required Dependencies

In [2]:
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader,TextLoader,DataFrameLoader
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
#
import os
import pandas as pd
from getpass import getpass
#
from datasets import load_dataset
#
import openai

## Initiatize openai

In [3]:
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

Enter your OpenAI API Key: ··········


## Load dataset

In [4]:
data = load_dataset("lukesjordan/worldbank-project-documents", split="train")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['project_id', 'document_text', 'document_type'],
    num_rows: 15710
})

In [5]:
df = data.to_pandas()
df.head()


,project_id,document_text,document_type
0,P095978,Page 1\n40113 \n \nPROJECT INFORMATION DOCUMEN...,APPROVAL
1,P009057,ï»¿ Second Small & Medium Scale Industry\nRepo...,APPROVAL
2,P113117,Page 1\nINTEGRATED SAFEGUARDS DATASHEET \nAPPR...,APPROVAL
3,P131202,1\n 2\n 3\n 4\n 5\n 6\n 7\n 8\n 9\n 10\n 11\n ...,APPROVAL
4,P119295,51759\n PROJECT INFORMATION DOCUMENT (PID)\n A...,APPROVAL


In [ ]:
print(df.iloc[4]['document_text'])

51759
 PROJECT INFORMATION DOCUMENT (PID)
 APPRAISAL STAGE

 Project Name Karnataka Wind Power Carbon Finance Project, India
 Region South Asia
 Sector SASDE
 Project ID P119295
 Borrower(s) NA
 Implementing Agency Acciona Wind Energy Pvt\. Ltd\. (AWEPL), a 100% subsidiary of
 the Spanish Acciona group of companies, i\.e\. Acciona Energia
 Internacional, S\.A\. and Acciona, S\.A\.
 Environment Category [ ] A [X] B [ ] C [ ] FI [ ] TBD (to be determined)
 Date PID Prepared 19 November 2009
 Date of Appraisal 26 October 2009
 Authorization
 Estimated Date of ERPA 21 December 2009
 Signing



I\. Country and Sector Background

The total installed power generation capacity in India at the end of the 10th Plan was
132329\.21MW which included 7760\.60MW from renewable sources of energy\. Keeping in view
the existing energy deficit of 11%, peak deficit of 12% and the envisaged growth of 9% in the
Eleventh Plan, the target of capacity addition in the Eleventh Plan period is an ambitious
78577M

In [24]:
df.columns

Index(['project_id', 'document_text', 'document_type'], dtype='object')

## Load the data from dataframe into Document using DataFrame loader

In [6]:
loader = DataFrameLoader(df,page_content_column="document_text")
data = loader.load()
print(len(data))

15710


In [26]:
data[0].metadata


{'project_id': 'P095978', 'document_type': 'APPROVAL'}

## Create a new processed document to include the source object to facilitate return documents as source of referrence

In [7]:
from langchain.docstore.document import Document
processed_data = []
for doc in data:
  processed_data.append(Document(page_content=doc.page_content, metadata={"source": str(doc.metadata)}))

print(len(processed_data))
print(len(data))

15710
15710


In [8]:
processed_data[0].metadata

{'source': "{'project_id': 'P095978', 'document_type': 'APPROVAL'}"}

In [9]:
processed_data[0].page_content

"Page 1\n40113 \n \nPROJECT INFORMATION DOCUMENT (PID) \n \nProject Name \nNepal Village Micro Hydro Carbon Offset Project \nRegion \nSouth Asia \nSector \nSASSD \nProject ID \nP095978 \nBorrower(s) \n \nImplementing Agency \nAlternative Energy Promotion Center \nEnvironment Category \n[] A [X ] B [ ] C [ ] FI [ ] TBD \nSafeguard Classification \n[]S\n1\n[X ] S\n2\n[\n]\nS\n3\n[\n]\nS\nF\n[\n]\nTBD \n \nDate PID Revised \nMay 7, 2007 \nDate of Appraisal \nAuthorization \nApril 27, 2007 \nEstimated Date of ERPA \nSignature \nJune 17, 2007 \n1\\. \nCountry and Sector Background \n \nNepal has vast hydro resources, which represent a source of potential wealth\\. Commercially \nexploitable hydropower generating potential is estimated to be about 43,000 MW\\. However, \ndespite this large potential, only about 600 MW has been developed so far\\. Based on the 2001 \ncensus, 40 percent of Nepal's households have access to electricity, but disparity in access is \nstark\\. While over 90 percen

## Setup embedding model

In [10]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.embeddings import HuggingFaceBgeEmbeddings
#
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings":True}
#
embeddings = HuggingFaceBgeEmbeddings(model_name=model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs,
                                      )


## Store data in vectorstore

In [ ]:
def split_list(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i + chunk_size]
    return input_list

In [ ]:
split_docs_chunked = split_list(data, 41000)

In [ ]:
split_docs_chunked

<generator object split_list at 0x7b7e86d9b760>

In [ ]:
from tqdm.notebook import tqdm
for split_docs_chunk in tqdm(split_docs_chunked):
    vectordb = Chroma.from_documents(
        documents=split_docs_chunk,
        embedding=embeddings,
        persist_directory="./chroma_db",
        collection_name="split_docs",
    )
    vectordb.persist()

0it [00:00, ?it/s]

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":1})


In [ ]:
from langchain.chains import RetrievalQA,RetrievalQAWithSourcesChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-16k",temperature=0.75)

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,

                                )





In [ ]:
query ="Please describe the project Karnataka Wind Power Carbon Finance Project"
qa.run(query=query)


'The Karnataka Wind Power Carbon Finance Project is a renewable energy project that involves the installation of a wind power plant in Karnataka, India. The project aims to reduce greenhouse gas emissions by generating clean electricity from wind energy. \n\nThe project consists of 18 wind turbines with a total capacity of 29.7 MW, spread over two locations in Davangere district. The electricity generated from the project is supplied to the state grid through a sub-station owned by Karnataka Power Transmission Corporation Limited. \n\nThe project was started by Bellary Wind Power Private Limited and later acquired by Acciona Wind Energy Private Limited (AWEPL), a subsidiary of the Spanish Acciona group of companies. The project has been operational since 2008 and has been delivering clean electricity to the grid. \n\nThe project is registered under the Clean Development Mechanism (CDM) of the United Nations Framework Convention on Climate Change (UNFCCC). It has received approval for t

In [ ]:
query ="What is the project Uruguay-Public Administration Reform about?"
qa(query)

{'query': 'What is the project Uruguay-Public Administration Reform about?',
 'result': 'The project "Uruguay-Public Administration Reform" aims to assist the government of Uruguay in modernizing and reducing the size of government agencies, introducing modern integrated financial management systems, and implementing an integrated approach to regulatory activities focused on eliminating impediments to private market efficiency. The project includes three components: downsizing and agency modernization, reform of the public financial management system, and deregulation to improve economic competitiveness. The project is designed to build on previous reform efforts and enhance the efficiency of the public sector and private sector in Uruguay.'}

## Parent Document Retriever
* Return full documents for smaller chunks lookup
* Return bigger chunks for smaller chunks loop

##  storage layer for parent documents

In [11]:
store = InMemoryStore()

## Create Parent and child text splitters

In [12]:
#create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
#
#create bigger chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
#

## 1. Retrieving full documents rather than chunks
In this mode, we want to retrieve the full documents.

This is good to use if the  initial full docs are not too big themselves and we are not going to return many of them

####  The vectorstore to use to index the child chunks

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=embeddings,
    persist_directory="./chroma_db",

)


#### Create the Full Document Retriever

In [ ]:
full_doc_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_text_splitter,
)

#### Add documents to the docstore and vectorstores.

**Parameters**

* documents – List of documents to add

* ids – Optional list of ids for documents. If provided should be the same length as the list of documents. Can provided if parent documents are already in the document store and you don’t want to re-add to the docstore. If not provided, random UUIDs will be used as ids.

* add_to_docstore – Boolean of whether to add documents to docstore. This can be false if and only if ids are provided. You may want to set this to False if the documents are already in the docstore and you don’t want to re-add them.

In [ ]:
MAX_BATCH_SIZE = 500
Num_Records = len(data)
no_of_batches = Num_Records // MAX_BATCH_SIZE
no_of_batches

31

In [ ]:
from tqdm.notebook import tqdm
for i in tqdm(range(0,len(data),MAX_BATCH_SIZE)):
  print(i)
  i_end = min(len(data),i+MAX_BATCH_SIZE)
  print(i_end)
  batch = data[i:i_end]
  full_doc_retriever.add_documents(batch, ids=None)


  0%|          | 0/158 [00:00<?, ?it/s]

0
100
100
200
200
300
300
400
400
500
500
600
600
700


ValueError: ignored

In [ ]:
list(store.yield_keys())

['133e1fd8-1854-49aa-8573-253bb466a581',
 'e7b68bd1-b231-46b7-9691-56f05a6557a5',
 '1b031797-b04a-4f2d-a45d-713830ce67b4',
 'af7e292a-a879-40e1-8f25-fa5e88d8c66e',
 '51c3ad12-18ab-439a-87cc-0b76b207ac7c',
 '55db4f21-c4fa-44c2-a279-cd749dd22703',
 '64e075f1-818d-46c8-88f0-433c67fd74da',
 'd42be499-5417-4486-a8c4-f86c0522f894',
 '5e50a20c-c371-4b71-94ae-aa6c5ef8f391',
 'b2740610-93fd-420d-bf2f-d416542442ec',
 'eb1b78ff-f38f-41fa-82d8-44856037a887',
 'ec326012-94cf-4fb7-a3d2-aec8207840a7',
 '337ef97c-1b93-4c25-9346-b20cd5e8d1c3',
 '49528cb2-9266-46e8-a4b3-2ff33a759bfe',
 '5d01f4b2-1ce8-4942-be8d-a534680a8a19',
 '143a6057-568c-4481-9241-29c0285bd340',
 'e6fe6e98-e00f-4fbd-a767-57fdc229e9fd',
 '4882a4b5-6c6b-48a2-bf3a-f9a0278933e7',
 '2488dd75-12a1-42ff-8a54-b700e12d97f2',
 'dfe64086-e43e-4dc8-9e39-dda82fd08713',
 '7647da87-5171-4e78-819d-2b6386e4aa4b',
 'f33b2784-8c35-402e-9d81-58516b6be6ba',
 '82ad785e-ed96-45ec-b9b5-14a8578a7374',
 '8cea7f08-95bd-4f01-9baf-1f5a7ad92973',
 '6b568904-73a6-

In [ ]:
len(list(store.yield_keys()))

600

## Retrive full documents

In [ ]:
sub_docs = vectorstore.similarity_search("Who started the project Karnataka Wind Power Carbon Finance Project?", k=5)

In [ ]:
len(sub_docs)

5

In [ ]:
print(sub_docs[0].page_content)


other project operators\. Carbon finance will help scale up and accelerate access to electricity by
 providing a connection fee subsidy (in the form of a connection coupon/voucher) to rural
 households in the concession areas in order to reduce the householdsÃÂ´ connection costs by
 making the payment of the connection charge affordable to the poor rural population\. It is


In [ ]:
print(sub_docs[1].page_content)

15\. The proposed Project is part of the World Bankâs comprehensive clean energy transition support
 program\. It mobilizes much-needed public financing to address the grid capacity constraints specifically
 targeted to sustainably increase the penetration of renewable energy\. These investments lay a strong


In [ ]:
print(sub_docs[1].page_content)

15\. The proposed Project is part of the World Bankâs comprehensive clean energy transition support
 program\. It mobilizes much-needed public financing to address the grid capacity constraints specifically
 targeted to sustainably increase the penetration of renewable energy\. These investments lay a strong


In [ ]:
print(sub_docs[2].page_content)

The Carbon Finance Project will incorporate lessons learned from the initial implementation 
results of the Power Development Project and the previous phases of the UNDP REDP project\. 
Recent experience in Nepal has demonstrated that delivery of infrastructure services to rural 
communities is more effective and sustainable when beneficiaries are able to actively participate


In [ ]:
print(sub_docs[3].page_content)

The Project will use carbon-linked results based payment scheme to support the implementation
 of the GoS plan to scale up and accelerate rural electrification through private concessionaires and
 other project operators\. Carbon finance will help scale up and accelerate access to electricity by
 providing a connection fee subsidy (in the form of a connection coupon/voucher) to rural


In [ ]:
print(sub_docs[4].page_content)

Micro- and SME finance (10%); Renewable energy (10%)
Project ID P077572
Borrower(s) GOVERNMENT OF CHHATTISGARH
Implementing Agency
Address STATE FOREST DEPT\., VILLAGE
FORESTRY INSTITUTIONS, PRI
Contact Person: Mr\. Ramprakash,
Principal Secretary, Forests
Tel: (0771) 221157
Fax: (0771) 331120
Email: drrscharma@yahoo\.com
Environment Category B
Date PID Prepared March 25, 2003


## Retrieving larger chunks

Sometimes, the full documents can be too big to want to retrieve them as is.

In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks.

We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

#### Create Vectorstore for retrieving larger chunks

In [13]:
vectordb = Chroma(
        embedding_function=embeddings,
        persist_directory="./chroma_db_parent",
        collection_name="split_parents",
    )

## Initialize Parent Document Retrievers

In [14]:
big_chunks_retrievr = ParentDocumentRetriever(
    vectorstore=vectordb,
    docstore=store,
    child_splitter=child_text_splitter,
    parent_splitter=parent_splitter,
   )

```file /usr/local/lib/python3.10/dist-packages/chromadb/api/segment.py:394, in SegmentAPI._upsert(self, collection_id, ids, embeddings, metadatas, documents)
    392 coll = self._get_collection(collection_id)
    393 self._manager.hint_use_collection(collection_id, t.Operation.UPSERT)
--> 394 validate_batch(
    395     (ids, embeddings, metadatas, documents),
    396     {"max_batch_size": self.max_batch_size},
    397 )
    398 records_to_submit = []
    399 for r in _records(
    400     t.Operation.UPSERT,
    401     ids=ids,
   (...)
    404     documents=documents,
    405 ):

File /usr/local/lib/python3.10/dist-packages/chromadb/api/types.py:377, in validate_batch(batch, limits)
    372 def validate_batch(
    373     batch: Tuple[IDs, Optional[Embeddings], Optional[Metadatas], Optional[Documents]],
    374     limits: Dict[str, Any],
    375 ) -> None:
    376     if len(batch[0]) > limits["max_batch_size"]:
--> 377         raise ValueError(
    378             f"Batch size {len(batch[0])} exceeds maximum batch size {limits['max_batch_size']}"
    379         )

```

ValueError: Batch size 373981 exceeds maximum batch size 41666


ValueError: Batch size 45535 exceeds maximum batch size 41666

## Add the processed data to the documentstore and vectorstore

In [15]:
MAX_BATCH_SIZE = 100

In [41]:
%%time
from tqdm.notebook import tqdm
for i in tqdm(range(600,len(processed_data[0:4000]),MAX_BATCH_SIZE)):
  print(f"start : {i}")
  i_end = min(len(processed_data),i+MAX_BATCH_SIZE)
  print(f"end: {i_end}")
  batch = processed_data[i:i_end]
  try:
    big_chunks_retrievr.add_documents(batch, ids=None)
  except ValueError as e:
    print(e)
    #ct = i_end // 2
    temp = batch
    #print(ct)
    big_chunks_retrievr.add_documents(temp[:50], ids=None)
    big_chunks_retrievr.add_documents(temp[50:], ids=None)
    continue
  vectordb.persist()
  print(f"Number of keys stored in the doctstore :{len(list(store.yield_keys()))}")

  0%|          | 0/34 [00:00<?, ?it/s]

start : 600
end: 700
Batch size 42829 exceeds maximum batch size 41666
start : 700
end: 800
Number of keys stored in the doctstore :43100
start : 800
end: 900
Number of keys stored in the doctstore :47442
start : 900
end: 1000
Batch size 44282 exceeds maximum batch size 41666
start : 1000
end: 1100
Number of keys stored in the doctstore :58500
start : 1100
end: 1200
Number of keys stored in the doctstore :63716
start : 1200
end: 1300
Number of keys stored in the doctstore :69123
start : 1300
end: 1400
Number of keys stored in the doctstore :73693
start : 1400
end: 1500
Number of keys stored in the doctstore :79339
start : 1500
end: 1600
Batch size 44080 exceeds maximum batch size 41666
start : 1600
end: 1700
Number of keys stored in the doctstore :91119
start : 1700
end: 1800
Number of keys stored in the doctstore :96438
start : 1800
end: 1900
Number of keys stored in the doctstore :101413
start : 1900
end: 2000
Number of keys stored in the doctstore :106119
start : 2000
end: 2100
Numb

## To add only first 100 documents to the vectorstore and coument store

In [ ]:
big_chunks_retrievr.add_documents(data[:100])


In [ ]:
vectordb.persist()

In [ ]:
big_chunks_retrievr.add_documents(data[100:200])

In [17]:
vectordb.persist()

In [18]:
len(list(store.yield_keys()))

31856

In [ ]:
len(list(store.yield_keys()))

10018

In [ ]:
vectordb.persist()
#
len(list(store.yield_keys()))

10018

## smaller chunks

In [19]:
sub_docs = vectordb.similarity_search("What is the project Uruguay-Public Administration Reform about?")

In [20]:
print(len(sub_docs)) #default 4 documents will be retrieved since value of k =4

4


In [21]:
print(sub_docs[0].page_content)


Report No\. PIC1062
Project Name Uruguay-Public Administration Reform Project (@)
Region Latin America Caribbean
Sector Public Sector Management
Project ID UYPA8172
Borrower Republic of Uruguay
Beneficiary Ministry of Economy
Financing Plan IBRD US$23 million
Government US$ 7 million
Tentative Appraisal Date TBD
Projected Board Date August 11, 1997


In [22]:
print(sub_docs[1].page_content)

Uruguay calls for improvements in the administration of general taxes
and customs; help in reducing social security tax evasion and improving
accounting of social security contributions; reductions in the public
sector deficit; and improvements in public sector management\. The
downsizing and agency modernization component and the PFM system reform


In [23]:
print(sub_docs[2].page_content)

sector deficit; and improvements in public sector management\. The
downsizing and agency modernization component and the PFM system reform
component of this project address these elements of the Bank's public
sector strategy in Uruguay\. In addition, Uruguay's legal framework for
productive activities is singled out for attention in the Bank's most
- 3 -


In [24]:
print(sub_docs[3].page_content)

Borrower Republic of Uruguay
Beneficiary Ministry of Economy
Financing Plan IBRD US$23 million
Government US$ 7 million
Tentative Appraisal Date TBD
Projected Board Date August 11, 1997
1\. Background\. Breaking with its traditional statim, Uruguay began
some time ago to reform the public sector\. Its problems with public
enterprises revolve around inefficient monopolistic systems\. The public


## Question 2

In [42]:
sub_docs = vectordb.similarity_search("Who is the Project Team Leader mentioned in the Report No: ICR00001193")

In [45]:
len(sub_docs)


4

In [44]:
print(sub_docs[0].page_content)


Project Team Leader: Paul G\. A\. Bance Bernard Harborne

 ICR Team Leader: Peter F\. B\. A\. Lafere

 ICR Primary Author: Peter F\. B\. A\. Lafere

 Felipe Jacome




 F\. Results Framework Analysis


 Project Development Objectives (from Project Appraisal Document)


In [46]:
print(sub_docs[1].page_content)


Manager/Manager:
Project Team Leader: Barbara Kasura Magezi Ndamira Denyse E\. Morin
ICR Team Leader: Barbara Kasura Magezi Ndamira
ICR Primary Author: Michael Christopher Jelenic
F\. Results Framework Analysis


In [47]:
print(sub_docs[2].page_content)


Follmer (Team Leader), Mr\. Dick Coppinger (Civil Engineer, Consultant), Ms\. Wacuka Ikua (Operations
Officer), Mr\. Dahir Warsame (Procurement Specialist), and Mr\. John Ogallo (Financial Management
Specialist)\. This report is based on information collected during that mission and material from the project
files\.


In [48]:
print(sub_docs[3].page_content)

Project Team Leader: Michel Audige

 ICR Team Leader: Ross Pavis

 REPUBLIC OF POLAND
 Szczecin-Swinoujscie Seaway and Port Modernization Project




 CONTENTS




Data Sheet
 A\. Basic Information
 B\. Key Dates
 C\. Ratings Summary
 D\. Sector and Theme Codes
 E\. Bank Staff
 F\. Results Framework Analysis
 G\. Ratings of Project Performance in ISRs
 H\. Restructuring
 I\. Disbursement Graph


## Bigger Chunks

In [51]:
bigger_chunks_retrieved_docs = big_chunks_retrievr.get_relevant_documents("What is the project Uruguay-Public Administration Reform about?")


In [53]:
len(bigger_chunks_retrieved_docs)


3

In [54]:
print(bigger_chunks_retrieved_docs[0].page_content)
print("*"*80)
print(bigger_chunks_retrieved_docs[1].page_content)
print("*"*80)
print(bigger_chunks_retrieved_docs[2].page_content)


Report No\. PIC1062
Project Name Uruguay-Public Administration Reform Project (@)
Region Latin America Caribbean
Sector Public Sector Management
Project ID UYPA8172
Borrower Republic of Uruguay
Beneficiary Ministry of Economy
Financing Plan IBRD US$23 million
Government US$ 7 million
Tentative Appraisal Date TBD
Projected Board Date August 11, 1997
1\. Background\. Breaking with its traditional statim, Uruguay began
some time ago to reform the public sector\. Its problems with public
enterprises revolve around inefficient monopolistic systems\. The public
sector is generally overexpanded, outdated, and needs to be
rationalized\. In addition to off-loading public enterprises and
downsizing the Central Government, there is a need to modernize core
government functions, including financial management, tax collections,
and customs and social security administration\. The 1985-89
administration began important reforms, including: elimination of
passenger service and reduction of uneconomic 

## Bigger Chunks -2

In [49]:
bigger_chunk_retrieved_docs = big_chunks_retrievr.get_relevant_documents("Who is the Project Team Leader mentioned in the Report No: ICR00001193")
print(len(bigger_chunk_retrieved_docs))


4


In [50]:
print(bigger_chunk_retrieved_docs[0].page_content)
print("*"*80)
print(bigger_chunk_retrieved_docs[1].page_content)
print("*"*80)
print(bigger_chunk_retrieved_docs[2].page_content)
print("*"*80)
print(bigger_chunk_retrieved_docs[3].page_content)

Outcomes: Moderately Unsatisfactory

Risk to Development Outcome: High

Bank Performance: Moderately Unsatisfactory

Borrower Performance: Moderately Satisfactory



C\.2 Detailed Ratings of Bank and Borrower Performance (by ICR)
 Bank Ratings Borrower Ratings
 Moderately Moderately
Quality at Entry: Government:
 Unsatisfactory Satisfactory
Quality of Moderately Implementing Moderately
Supervision: Satisfactory Agency/Agencies: Satisfactory




 6
Overall Bank Moderately Overall Borrower Moderately
Performance: Unsatisfactory Performance: Satisfactory


C\.3 Quality at Entry and Implementation Performance Indicators
 Implementation QAG Assessments
 Indicators Rating
 Performance (if any)

 Potential Problem
 Quality at Entry
Project at any time Yes None
 (QEA):
(Yes/No):

 Problem Project at any Quality of
 Yes None
time (Yes/No): Supervision (QSA):

DO rating before Moderately
Closing/Inactive status: Satisfactory




D\. Sector and Theme Codes
 Original Actual

Sector Code (as % of 

## Retrivel Question Answer Chain - Without Source Document

In [56]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-16k",temperature=0.75)
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=big_chunks_retrievr,

                                )

## Retrivel Question Answer Chain - With Source Document

In [55]:
from langchain.chains import RetrievalQAWithSourcesChain
qa_with_source = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=big_chunks_retrievr,

                                )

## Query Information Retrieval

#### Query 1

In [57]:
qa("What is the project Uruguay-Public Administration Reform about?")

{'query': 'What is the project Uruguay-Public Administration Reform about?',
 'result': 'The project "Uruguay-Public Administration Reform" aims to reform the public sector in Uruguay. The goals of the project include off-loading public enterprises, downsizing the Central Government, and modernizing core government functions such as financial management, tax collections, and customs and social security administration. The project also seeks to increase private sector participation in state monopolies and reduce regulatory and reporting requirements. The project is designed to build on previous reforms and serve as a basis for economic recovery in Uruguay.'}

In [58]:
qa_with_source("What is the project Uruguay-Public Administration Reform about?")

{'question': 'What is the project Uruguay-Public Administration Reform about?',
 'answer': "The project Uruguay-Public Administration Reform is about implementing reforms to boost economic growth, improve competitiveness, and enhance the provision of public and social services in Uruguay. The project aims to modernize core government functions, such as financial management, tax collections, and customs and social security administration. It also seeks to reduce regulatory and reporting requirements and promote private sector participation in state monopolies. The project is designed to build on the government's previous reform efforts and support its goals of improving the performance of the central government and reducing impediments to private markets.\n",
 'sources': "{'project_id': 'P008172', 'document_type': 'APPROVAL'}"}

####Query2

In [59]:
qa("Who started the project Karnataka Wind Power Carbon Finance Project?")

{'query': 'Who started the project Karnataka Wind Power Carbon Finance Project?',
 'result': 'There is no information provided in the given context about who started the Karnataka Wind Power Carbon Finance Project.'}

In [60]:
qa_with_source("Who started the project Karnataka Wind Power Carbon Finance Project?")

{'question': 'Who started the project Karnataka Wind Power Carbon Finance Project?',
 'answer': 'The document does not provide information about who started the Karnataka Wind Power Carbon Finance Project.\n',
 'sources': ''}

#### Query3

In [ ]:
qa("Contact Point for the project Karnataka Wind Power Carbon Finance Project?")


{'query': 'Contact Point for the project Karnataka Wind Power Carbon Finance Project?',
 'result': 'The contact point for the Karnataka Wind Power Carbon Finance Project is Acciona Wind Energy Pvt. Ltd. (AWEPL), a subsidiary of the Spanish Acciona group of companies.'}

In [65]:
qa_with_source("Who is the point of contact for the project Karnataka Wind Power Carbon Finance Project?")

{'question': 'Who is the point of contact for the project Karnataka Wind Power Carbon Finance Project?',
 'answer': 'The point of contact for the project Karnataka Wind Power Carbon Finance Project is not mentioned in the provided information.\n',
 'sources': ''}

#### Query 3

In [ ]:
qa("What was the Date of Appraisal for the Karnataka Wind Power Carbon Finance Project?")

{'query': 'What was the Date of Appraisal for the Karnataka Wind Power Carbon Finance Project?',
 'result': 'The Date of Appraisal for the Karnataka Wind Power Carbon Finance Project was October 26, 2009.'}

In [64]:
qa_with_source("Date of Appraisal for the Karnataka Wind Power Carbon Finance Project?")

{'question': 'Date of Appraisal for the Karnataka Wind Power Carbon Finance Project?',
 'answer': 'The date of Appraisal for the Karnataka Wind Power Carbon Finance Project is October 26, 2009.\n',
 'sources': "{'project_id': 'P119295', 'document_type': 'APPROVAL'}"}

## Query 4

In [ ]:
qa("What is the Estimated Date of ERPA for the Karnataka Wind Power Carbon Finance Project?")


{'query': 'What is the Estimated Date of ERPA for the Karnataka Wind Power Carbon Finance Project?',
 'result': 'The estimated date of ERPA (Emission Reduction Purchase Agreement) for the Karnataka Wind Power Carbon Finance Project is 21 December 2009.'}

In [63]:
qa_with_source("What is the Estimated Date of ERPA for the Karnataka Wind Power Carbon Finance Project?")

{'question': 'What is the Estimated Date of ERPA for the Karnataka Wind Power Carbon Finance Project?',
 'answer': 'The estimated date of ERPA for the Karnataka Wind Power Carbon Finance Project is 21 December 2009.\n',
 'sources': "{'project_id': 'P119295', 'document_type': 'APPROVAL'}"}

#### Query 5

In [ ]:
qa("What is the objective of the Karnataka Wind Power Carbon Finance Project?")


{'query': 'What is the objective of the Karnataka Wind Power Carbon Finance Project?',
 'result': 'The objective of the Karnataka Wind Power Carbon Finance Project is to contribute to sustainable development by providing renewable power to the Karnataka State electricity grid and reduce greenhouse gas emissions caused by reliance on fossil fuels. Additionally, the project aims to reduce global emissions of carbon dioxide.'}

In [66]:
qa_with_source("What is the objective of the Karnataka Wind Power Carbon Finance Project?")

{'question': 'What is the objective of the Karnataka Wind Power Carbon Finance Project?',
 'answer': 'The objective of the Karnataka Wind Power Carbon Finance Project is to contribute to sustainable development by providing renewable power to the Karnataka State electricity grid and reduce greenhouse gas emissions caused by reliance on fossil fuels. The project also aims to reduce global emissions of carbon dioxide.\n',
 'sources': "{'project_id': 'P119295', 'document_type': 'APPROVAL'}"}

## Query 6

In [ ]:
qa("What has been mentioned about Sustainability in the Karnataka Wind Power Carbon Finance Project?")



{'query': 'What has been mentioned about Sustainability in the Karnataka Wind Power Carbon Finance Project?',
 'result': "The project activity in the Karnataka Wind Power Carbon Finance Project aims to achieve sustainable development by:\n- Substituting electricity generated using conventional fossil fuel with wind energy-based power\n- Mitigating the emission of greenhouse gases, specifically CO2, as wind is a renewable source of energy\n- Conserving coal and other non-renewable natural resources\n- Contributing to the growth of the wind energy sector in India\n- Increasing the share of renewable energy in the regional grid\n- Reducing pollutants associated with other power plants, such as SOX and NOX\n- Contributing to reducing power shortage in the state of Karnataka\n- Generating local employment and economic activity in the project area\n- Helping to bridge India's energy deficit"}

In [67]:
qa_with_source("What has been mentioned about Sustainability in the Karnataka Wind Power Carbon Finance Project?")

{'question': 'What has been mentioned about Sustainability in the Karnataka Wind Power Carbon Finance Project?',
 'answer': "The Karnataka Wind Power Carbon Finance Project mentions that the project aims to mobilize climate financing, specifically international performance-based financing in sequestering greenhouse gases and carbon. This financing will enable them to scale up investments and ensure the sustainability of the project's activities in the area.\n",
 'sources': "{'project_id': 'P173830', 'document_type': 'APPROVAL'}"}

## Query 7

In [ ]:
qa("What was the brokerage service offered to ex-soldiers by Pilot Emergency Labor Redeployment Project (PELRP)?")

{'query': 'What was the brokerage service offered to ex-soldiers by Pilot Emergency Labor Redeployment Project (PELRP)?',
 'result': 'The specific details of the brokerage service offered to ex-soldiers by the Pilot Emergency Labor Redeployment Project (PELRP) are not provided in the given context.'}

In [68]:
qa_with_source("What was the brokerage service offered to ex-soldiers by Pilot Emergency Labor Redeployment Project (PELRP)?")

{'question': 'What was the brokerage service offered to ex-soldiers by Pilot Emergency Labor Redeployment Project (PELRP)?',
 'answer': 'The brokerage service offered to ex-soldiers by Pilot Emergency Labor Redeployment Project (PELRP) is not mentioned in the provided information.\n',
 'sources': 'N/A'}

In [69]:
qa("How many ex-soldiers were benefittedby the Pilot Emergency Labor Redeployment Project (PELRP)?")

{'query': 'How many ex-soldiers were benefittedby the Pilot Emergency Labor Redeployment Project (PELRP)?',
 'result': 'There is no information provided in the given context about the number of ex-soldiers who were benefited by the Pilot Emergency Labor Redeployment Project (PELRP).'}

## Query 8

In [70]:
qa("Who is the Project Team Leader mentioned in the Report No: ICR00001193")

{'query': 'Who is the Project Team Leader mentioned in the Report No: ICR00001193',
 'result': 'The Project Team Leader mentioned in the report is Paul G. A. Bance.'}

In [71]:
qa_with_source("Who is the Project Team Leader mentioned in the Report No: ICR00001193")

{'question': 'Who is the Project Team Leader mentioned in the Report No: ICR00001193',
 'answer': 'The Project Team Leader mentioned in the Report No: ICR00001193 is Paul G. A. Bance.\n',
 'sources': '0-pl'}

####Query 10

In [72]:
qa("Summarize the Report no :ICR00001193")

{'query': 'Summarize the Report no :ICR00001193',
 'result': "The report, ICR00001193, provides an assessment of a project's outcomes and impacts in the education sector. The project successfully implemented educational planning instruments, such as PROMEs and PENs, which guided municipal investment in education. Gender equity was also integrated into the education agenda. Additionally, the project produced relevant results in designing a strategy for secondary education, linking it to economic activities. There was increased dialogue and capacity strengthening between the government and public universities. Beneficiary surveys and workshops showed widespread knowledge of the project's objectives, satisfaction with implementation, and recommendations for improvement. Overall, the risk to development outcome was rated as moderate. The report also includes information on the project's task team members, staff time and cost, and beneficiary survey results."}

In [73]:
qa_with_source("Summarize the Report no :ICR00001193")

{'question': 'Summarize the Report no :ICR00001193',
 'answer': "The report no: ICR00001193 discusses the unintended outcomes and impacts of the project, including the use of PROMEs and PENs as educational planning instruments for municipalities, the inclusion of gender equity in the education agenda, the linkage between secondary education and economic activities, the dialogue and capacity strengthening between the government and public universities, and the opportunity for public universities to receive additional funds for quality improvement activities. The report also mentions the findings of beneficiary surveys and stakeholder workshops, which showed widespread knowledge of the project's objectives, satisfaction with its implementation, and recommendations for improvement. The report concludes with an assessment of the risk to development outcome, rating it as moderate.\n",
 'sources': "{'project_id': 'P006204', 'document_type': 'APPROVAL'}"}

### Query 11

In [74]:
qa_with_source("What is the project Uruguay-Public Administration Reform about?")


{'question': 'What is the project Uruguay-Public Administration Reform about?',
 'answer': 'The project "Uruguay-Public Administration Reform" aims to support reforms to boost economic growth, improve the competitiveness of the Uruguayan economy, and improve the provision of public and social services. The project focuses on areas such as public sector management, financial management, tax collections, customs, and social security administration. It also includes downsizing the central government, off-loading public enterprises, and modernizing core government functions. The project is designed to build on the groundwork already laid by the Uruguayan government in improving the performance of the central government and reducing impediments to private markets.\n',
 'sources': "{'project_id': 'P008172', 'document_type': 'APPROVAL'}, {'project_id': 'P078726', 'document_type': 'APPROVAL'}"}

In [75]:
qa("What is the project Uruguay-Public Administration Reform about?")

{'query': 'What is the project Uruguay-Public Administration Reform about?',
 'result': "The Uruguay-Public Administration Reform project is focused on reforming the public sector in Uruguay. The goal is to modernize core government functions such as financial management, tax collections, customs, and social security administration. The project aims to improve the efficiency and effectiveness of the public sector and reduce obstacles to private markets. It also includes downsizing the central government and off-loading public enterprises. The project is part of Uruguay's broader efforts to improve its economic growth, competitiveness, and provision of public and social services."}